In [100]:
#importar
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from statistics import mean
from numpy.linalg import norm
from matplotlib.colors import ListedColormap
import random
import time
import timeit

In [2]:
#importação da base de dados
#os dados estão disponíveis abertamente em http://archive.ics.uci.edu/ml/datasets/Covertype

file = "covtype.data"

#nome dos atributos
#estamos descartando os atributos que descrevem o tipo do solo
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache', 'Cover_Type']

#de 0 a 13 são os atributos listados acima, e 54 é a classificação
usecols = list(range(0, 14)) + [54]

#especifico o tipo de alguns parametros(os que não são simplesmente numéricos)
dtype = {'Cover_Type': 'category', 'Wilderness_Area_Rawah' : bool, 'Wilderness_Area_Neota' : bool, 'Wilderness_Area_Comanche' : bool, 'Wilderness_Area_Cache' : bool}

#lê o arquivo num pandas.dataframe
dataset = pandas.read_csv(file, header = None, usecols = usecols, names = names, dtype = dtype)

#adicionando uma coluna adicional para sintetizar os 4 boleanos que representam a Wilderness_area. 
#para uma única instância, somente um dos 4 booleanos pode ser verdadeiro, logo eles, em realidade, funcionam como uma categorização
new_column = pandas.Series([1 if dataset['Wilderness_Area_Rawah'][i] else 
                            2 if dataset['Wilderness_Area_Neota'][i] else
                            3 if dataset['Wilderness_Area_Comanche'][i] else
                            4 for i in range(len(dataset.index)) ], dtype="category")
#elimina as colunas reduzidas
dataset = dataset.drop(columns=['Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache'])
#insere nova coluna na posição 10
dataset.insert(loc = 10, column = 'Wilderness_Area', value = new_column)

#atualiza names para refletir a mudança acima
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Cover_Type']

print("Dataframe com %d exemplares e %d atributos importado com sucesso" % (dataset.values.shape[0], dataset.values.shape[1]))

Dataframe com 581012 exemplares e 12 atributos importado com sucesso


In [3]:
#test options and evaluation metric
seed = 7
scoring = 'accuracy'
#os atributos para treino em X e a classificação correta em Y
X = dataset.values[:, 0:11]
Y = dataset.values[:, 11]

In [93]:
array = dataset.values
X = array[:, 0:11]
Y = array[:, 11]


#preprocessa dataset
#estava preprocessando, mas não fazia diferença
#X_scale_temp = preprocessing.scale(X[:, 0:10])
#X_scaled = np.append(X_scale_temp, X[:, 10].reshape(-1, 1), axis = 1)

#separamos um pedaço do dataset para usarmos de varificação depois
validation_size = 0.2
seed = 7
#estratificado por Y
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size = validation_size, stratify = Y)#, random_state = seed**2)

print(X.shape, X_train.shape, X_validation.shape)
X_all = np.append(X_train, X_validation, axis = 0)
Y_all = np.append(Y_train, Y_validation, axis = 0)

print(X.shape, X_train.shape, X_validation.shape, X_all.shape)
print(Y.shape, Y_train.shape, Y_validation.shape, Y_all.shape)



(581012, 11) (464809, 11) (116203, 11)
(581012, 11) (464809, 11) (116203, 11) (581012, 11)
(581012,) (464809,) (116203,) (581012,)


In [98]:
#usamos um 10-fold para estimar a acurácia do método
def EstimarAcuraciaMetodo(metodo, k = 10):
    kfold = model_selection.KFold(n_splits = k, shuffle = True, random_state = 121)#, random_state = seed)
    cv_results = model_selection.cross_validate(metodo, X, Y, cv = kfold, scoring = scoring)
    print(cv_results)
    msg = "%s média entre os %d folds: %f, com desvio padrão de %f ,em " % (scoring, k, cv_results['test_score'].mean(), cv_results['test_score'].std())
    print(msg)
    msg2 = "tempo médio de treino: %f, para %d samples por fit" % (cv_results['fit_time'].mean(), (len(X_train) // k) * (k-1))
    print(msg2)

# Full Decision Tree 

In [99]:
dtc = DecisionTreeClassifier()
EstimarAcuraciaMetodo(dtc)

{'fit_time': array([7.24382329, 7.16426277, 7.01238894, 7.14673424, 7.39530301,
       7.16332293, 8.10411382, 8.58672857, 7.34939957, 6.99550843]), 'score_time': array([0.13495636, 0.11695266, 0.10936713, 0.12439513, 0.10895491,
       0.09376311, 0.20850873, 0.15769482, 0.10936761, 0.09974313]), 'test_score': array([0.93139651, 0.93120719, 0.93034543, 0.93239359, 0.93161908,
       0.92974303, 0.92831449, 0.93118879, 0.93084456, 0.93001842]), 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}
accuracy média entre os 10 folds: 0.930707, com desvio padrão de 0.001091 ,em 
tempo médio de treino: 7.416159, para 418320 samples por fit


In [108]:
#verificamos a acurácia do método aplicando-o sobre o set de validação que separamos anteriormente
dtc3 = DecisionTreeClassifier()
t0 = time.process_time()
dtc3.fit(X_train, Y_train)
tf = time.process_time()
print("Training time in seconds, for %d samples: %f" % (len(X_train), tf - t0))
predictions = dtc3.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
reshape = X_validation[0].reshape(1, -1)
N = 1000
print("Tempo médio para uma predição: %f segundos" % (timeit.timeit("dtc3.predict(reshape)", "from __main__ import dtc3, reshape", number = N) / N))

Training time in seconds, for 464809 samples: 6.859375
0.9276008364672168
[[39317  2773     4     0    42     6   226]
 [ 2675 53270   212     4   308   152    40]
 [    5   237  6475    70    21   343     0]
 [    0     1    68   450     0    30     0]
 [   41   281    25     0  1538    13     1]
 [   12   171   316    35     7  2932     0]
 [  260    34     0     0     0     0  3808]]
             precision    recall  f1-score   support

          1       0.93      0.93      0.93     42368
          2       0.94      0.94      0.94     56661
          3       0.91      0.91      0.91      7151
          4       0.81      0.82      0.81       549
          5       0.80      0.81      0.81      1899
          6       0.84      0.84      0.84      3473
          7       0.93      0.93      0.93      4102

avg / total       0.93      0.93      0.93    116203

Tempo médio para uma predição: 0.000033 segundos


# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11)
AvaliarMetodo(dtc)

In [112]:
#verificamos a acurácia do método aplicando-o sobre o set de validação que separamos anteriormente
knn = KNeighborsClassifier(n_neighbors = 11)
t0 = time.process_time()
knn.fit(X_train, Y_train)
tf = time.process_time()
print("Training time in seconds, for %d samples: %f" % (len(X_train), tf - t0))
predictions = knn.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
reshape = X_validation[0].reshape(1, -1)
N = 1000
print("Tempo médio para uma predição: %f segundos" % (timeit.timeit("knn.predict(reshape)", "from __main__ import knn, reshape", number = N) / N))

Training time in seconds, for 464809 samples: 2.359375
0.95926955414232
[[40503  1720     4     0    28     2   111]
 [ 1346 54985   111     1   125    74    19]
 [    2   121  6868    30     1   129     0]
 [    0     4    92   414     0    39     0]
 [   21   209    23     0  1640     6     0]
 [    2   122   211    11     5  3122     0]
 [  147    16     0     0     1     0  3938]]
             precision    recall  f1-score   support

          1       0.96      0.96      0.96     42368
          2       0.96      0.97      0.97     56661
          3       0.94      0.96      0.95      7151
          4       0.91      0.75      0.82       549
          5       0.91      0.86      0.89      1899
          6       0.93      0.90      0.91      3473
          7       0.97      0.96      0.96      4102

avg / total       0.96      0.96      0.96    116203

Tempo médio para uma predição: 0.000431 segundos


# KNN with three attributes

In [117]:
#verificamos a acurácia do método aplicando-o sobre o set de validação que separamos anteriormente

#seleciono os tres parametros mais relevantes, baseado na minha exploração dos dados
X_3 = np.zeros((dataset.values.shape[0], 3))
X_3[:, 0] = dataset['Elevation']
X_3[:, 1] = dataset['Horizontal_Distance_To_Roadways']
X_3[:, 2] = dataset['Horizontal_Distance_To_Fire_Points']

#separamos um pedaço do dataset para usarmos de varificação depois
validation_size = 0.2
seed = 7
#estratificado por Y
X_train_3, X_validation_3, Y_train_3, Y_validation_3 = model_selection.train_test_split(X_3, Y, test_size = validation_size, stratify = Y)




knn3 = KNeighborsClassifier(n_neighbors = 5)
t0 = time.process_time()
knn3.fit(X_train_3, Y_train_3)
tf = time.process_time()
print("Training time in seconds, for %d samples: %f" % (len(X_train_3), tf - t0))
predictions = knn3.predict(X_validation_3)
print(accuracy_score(Y_validation_3, predictions))
print(confusion_matrix(Y_validation_3, predictions))
print(classification_report(Y_validation_3, predictions))
reshape = X_validation_3[0].reshape(1, -1)
N = 1000
print("Tempo médio para uma predição: %f segundos" % (timeit.timeit("knn3.predict(reshape)", "from __main__ import knn3, reshape", number = N) / N))

Training time in seconds, for 464809 samples: 1.343750
0.818662168790823
[[34468  7300    29     0    47    15   509]
 [ 6569 48671   677     0   416   290    38]
 [   33   874  5495   128    17   604     0]
 [    0     3   227   256     0    63     0]
 [  112   771    40     0   970     6     0]
 [   19   450   883    38    10  2073     0]
 [  806    98     0     0     0     0  3198]]
             precision    recall  f1-score   support

          1       0.82      0.81      0.82     42368
          2       0.84      0.86      0.85     56661
          3       0.75      0.77      0.76      7151
          4       0.61      0.47      0.53       549
          5       0.66      0.51      0.58      1899
          6       0.68      0.60      0.64      3473
          7       0.85      0.78      0.82      4102

avg / total       0.82      0.82      0.82    116203

Tempo médio para uma predição: 0.000349 segundos
